In [40]:
import sys, importlib
importlib.reload(sys.modules['initialize_functions'])
importlib.reload(sys.modules['request_functions'])

ckpt-21


<module 'request_functions' from 'G:\\Users\\gui20\\python_projects\\ANPR_FINAL\\2.system\\request_functions.py'>

In [29]:
import json

from initialize_functions import *
import initialize_functions as init_fun

from request_functions import *
import request_functions as req_fun

In [3]:
def monitoring_spot():
    while True:
        enter_image_list = os.listdir(paths['IMGS_PARKING_SPOTS'])
        if enter_image_list:
            prepared_IMG = prepareIMG(os.path.join(paths['IMGS_PARKING_SPOTS'], enter_image_list[0]))
            ocr_res = ocr(prepared_IMG[0], prepared_IMG[1], 0.2, 52)
            plate = format_ocr(ocr_res)
            print("VAGAS - [VAGA: " + enter_image_list[0].replace('.jpg','') + " PLACA: " + plate + "]")

In [ ]:
def monitoring_spot():
    while True:
        enter_image_list = os.listdir(paths['IMGS_PARKING_SPOTS'])
        if enter_image_list:
            prepared_IMG = prepareIMG(os.path.join(paths['IMGS_PARKING_SPOTS'], enter_image_list[0]))
            ocr_res = ocr(prepared_IMG[0], prepared_IMG[1], 0.2, 52)
            plate = format_ocr(ocr_res)
            print("VAGAS - [VAGA: " + enter_image_list[0].replace('.jpg','') + " PLACA: " + plate + "]")

            #altera status da vaga para disponível caso não seja identificada uma placa na imagem
            if not plate:
                setParkingSpotAvailable(enter_image_list[0].replace('.jpg',''))
                continue

            spotID = enter_image_list[0]
            plateID =  json.loads(checkPlate(plate))['id']

            if plateID != spotID:
                #setParkingSpotAvailableFalse(spotID, plateID)
                print('diferente')
                continue

            print('igual')

In [4]:
monitoring_spot()

VAGAS - [VAGA: 1 PLACA: FBY2860]
VAGAS - [VAGA: 1 PLACA: FBY2860]


KeyboardInterrupt: 

In [19]:
checkSpot(1)

'{"id":1,"name":"p1","available":true,"plate":null}'

In [15]:
checkPlate('FBY2860')

'{"id":2,"plateNumber":"FBY2860"}'

In [17]:
def setParkingSpotAvailableFalse(spot_id, plate_id):
    url = 'https://tcc-parking-iot.herokuapp.com/parking-spots/'+str(spot_id)
    myobj = {
        "available" : "false",
        "plate" : {
            "id" : plate_id
        }
    }
    x = requests.put(url, json = myobj)
    return x.text

In [21]:
setParkingSpotAvailableFalse(1, 2)

'{"timestamp":"2022-11-15T18:57:26.551+00:00","status":500,"error":"Internal Server Error","path":"/parking-spots/1"}'

In [30]:
while True:
    enter_image_list = os.listdir(paths['IMGS_PARKING_SPOTS'])
    if enter_image_list:
        prepared_IMG = prepareIMG(os.path.join(paths['IMGS_PARKING_SPOTS'], enter_image_list[0]))
        ocr_res = ocr(prepared_IMG[0], prepared_IMG[1], 0.2, 52)
        plate = format_ocr(ocr_res)
        print("VAGAS - [VAGA: " + enter_image_list[0].replace('.jpg','') + " PLACA: " + plate + "]")
        
        #altera status da vaga para disponível caso não seja identificada uma placa na imagem
        if not plate:
            setParkingSpotAvailable(enter_image_list[0].replace('.jpg',''))
            continue
        
        spotID = enter_image_list[0]
        plateID =  json.loads(checkPlate(plate))['id']

        if plateID != spotID:
            #setParkingSpotAvailableFalse(spotID, plateID)
            print('diferente')
            continue
            
        print('igual')

VAGAS - [VAGA: 1 PLACA: FBY2860]
diferente
VAGAS - [VAGA: 1 PLACA: FBY2860]
diferente


KeyboardInterrupt: 

In [39]:
url = 'https://tcc-parking-iot.herokuapp.com/parking-spots/1'
myobj = {
        "name": "p1",
        "available": "false",
        "plate":{
            "id":1
        }
    }
x = requests.put(url, json = myobj)
x.text

'{"id":1,"name":"p1","available":false,"plate":{"id":1,"plateNumber":"abc-1234"}}'

In [38]:
createNewParkingSpot('p3')

'{"id":2,"name":"p2","available":true,"plate":null}'